In [46]:
import pandas as pd

In [47]:
share_energy_df = pd.read_csv('CSVS\\01 renewable-share-energy.csv')
production_df = pd.read_csv('CSVS\\03 modern-renewable-prod.csv')
consumption_df = pd.read_csv('CSVS\\02 modern-renewable-energy-consumption.csv')

solar_cap_df = pd.read_csv('CSVS\\13 installed-solar-PV-capacity.csv')
solar_cap_df

,Entity,Code,Year,Solar Capacity
0,Africa,NaN,1996,0.001112
1,Africa,NaN,1997,0.003137
2,Africa,NaN,1998,0.005162
3,Africa,NaN,1999,0.008199
4,Africa,NaN,2000,0.010927
...,...,...,...,...
1654,World,OWID_WRL,2017,390.206530
1655,World,OWID_WRL,2018,483.011800
1656,World,OWID_WRL,2019,584.685700
1657,World,OWID_WRL,2020,710.281100


In [48]:
merged_df = production_df.merge(consumption_df , on = ['Entity', 'Year', 'Code'], how='outer')
merged_df = merged_df.merge(share_energy_df , on = ['Entity', 'Year', 'Code'], how='outer')
merged_df = merged_df.merge(solar_cap_df, on = ['Entity', 'Year', 'Code'], how='outer')

In [49]:
# Normalización de los datos y creación de las tablas

In [50]:
# Crear la tabla de países/regiones sin duplicados
countries_df = merged_df[['Entity', 'Code']].drop_duplicates().reset_index(drop=True)
countries_df['country_id'] = countries_df.index + 1
countries_df = countries_df[['country_id', 'Entity', 'Code']]

In [51]:
# Crear el DataFrame de continentes utilizando diccionario por registro
data_continentes = [
    {'Continent': 'Africa', 'continent_id': 1},
    {'Continent': 'North America', 'continent_id': 2},
    {'Continent': 'South America', 'continent_id': 3},
    {'Continent': 'Oceania', 'continent_id': 4},
    {'Continent': 'Europe', 'continent_id': 5},
    {'Continent': 'Asia', 'continent_id': 6},
    {'Continent': 'Regions', 'continent_id': 7},
    {'Continent': 'Other', 'continent_id': 8}
]

# Crear el DataFrame
continentes_df = pd.DataFrame(data_continentes)
continentes_df = continentes_df[['continent_id', 'Continent']]

# Usamos un csv con los países y su respectivo continente
continentes_paises_df = pd.read_csv('CSVS\\18 Countries by continents.csv')
continentes_paises_df = continentes_paises_df.rename(columns={'Country':'Entity'})

# Asignar el continente al país usando el csv
countries_df = countries_df.merge(continentes_paises_df, on='Entity', how='left')

# Confirmamos los países africanos 54/54 + ['Reunion', 'Saint Helena', 'Western Sahara']
africa_countries = [
    "Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde", "Cameroon",
    "Central African Republic", "Chad", "Comoros", "Democratic Republic of the Congo", "Cote d'Ivoire",
    "Djibouti", "Egypt", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea",
    "Guinea-Bissau", "Equatorial Guinea", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi",
    "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria",
    "Republic of the Congo", "Rwanda", "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone",
    "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia",
    "Zimbabwe", "Reunion", "Saint Helena", "Western Sahara"
]
countries_df.loc[countries_df['Entity'].isin(africa_countries), 'Continent'] = 'Africa'

# Confirmamos los países 45/49 + ["Faeroe Islands", "Gibraltar", "USSR"] 47/49
europe_countries = [
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", 
    "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", 
    "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", 
    "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", 
    "Poland", "Portugal", "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City", "Armenia", "Azerbaijan", 
    "Georgia", "Turkey", "Faeroe Islands", "Gibraltar", "USSR"
]
countries_df.loc[countries_df['Entity'].isin(europe_countries), 'Continent'] = 'Europe'

# Confirmamos los países de Oceania 10/14 + ["American Samoa","Cook Islands","Guam","New Caledonia","Niue","French Polynesia"] 16/14
oceania_countries = [
    "Australia", "Fiji", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "New Zealand",
    "Palau", "Papua New Guinea", "Samoa", "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu", "American Samoa", "Cook Islands", "Guam", "New Caledonia", "Niue", "French Polynesia"
]
countries_df.loc[countries_df['Entity'].isin(oceania_countries), 'Continent'] = 'Oceania' #Falta palau, tuvalu, Marshall islands y micronesia

# Confirmamos los países de Asia 49/49 + ["Macao", "Timor"] 51/49
asia_countries = [
    "Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Cambodia", 
    "China", "Cyprus", "Georgia", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", 
    "Kazakhstan", "Kuwait", "Kyrgyzstan", "Laos", "Lebanon", "Malaysia", "Maldives", "Mongolia", 
    "Myanmar", "Nepal", "North Korea", "Oman", "Pakistan", "Palestine", "Philippines", "Qatar", 
    "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Tajikistan", "Thailand", 
    "Timor-Leste", "Turkey", "Turkmenistan", "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen", "Macao", "Timor"
]
countries_df.loc[countries_df['Entity'].isin(asia_countries), 'Continent'] = 'Asia'

# Confirmamos los países que hacen parte de North America + ["Bermuda", "Greenland", 'Saint Pierre and Miquelon'] (26)
north_america_countries = ["Bermuda", "Greenland", "Saint Pierre and Miquelon"]
countries_df.loc[countries_df['Entity'].isin(north_america_countries), "Continent"] = "North America"

# Confirmamos los países que hacen parte de South America (Incluyendo islas del caribe)
south_america_countries = ["Aruba","British Virgin Islands","Cayman Islands","Guadeloupe","Martinique","Montserrat","Puerto Rico","Turks and Caicos Islands","United States Virgin Islands","Falkland Islands", "French Guiana"]
countries_df.loc[countries_df['Entity'].isin(south_america_countries), "Continent"] = "South America"

# Other categories
other_categories = ["CIS (BP)", "G20 (Ember)", "G7 (Ember)", "High-income countries", "Low-income countries", "Lower-middle-income countries", "Non-OECD (BP)", "OECD (BP)", "OECD (Ember)", "Upper-middle-income countries", "World"]
countries_df.loc[countries_df['Entity'].isin(other_categories), 'Continent'] = 'Other'

# Regions
regions = [
    "Africa", "Africa (BP)", "Africa (Ember)", "Asia", "Asia (Ember)",
    "Asia Pacific (BP)", "Central America (BP)", "Eastern Africa (BP)", "Europe",
    "Europe (BP)", "Europe (Ember)", "European Union (27)",
    "European Union (27) (Ember)", "Latin America and Caribbean (Ember)",
    "Middle Africa (BP)", "Middle East (BP)", "North America",
    "North America (BP)", "North America (Ember)", "Oceania", "Oceania (Ember)",
    "South America", "South and Central America (BP)", "Western Africa (BP)"
]
countries_df.loc[countries_df['Entity'].isin(regions), 'Continent'] = 'Regions'

In [52]:
# Crear una nueva columna 'id_continente' y asignar el continente
countries_df = countries_df.merge(continentes_df, on = 'Continent', how = 'left')
countries_df = countries_df.drop(columns='Continent')
countries_df = countries_df.reset_index(drop=True)

In [53]:
# Crear la tabla de años sin duplicados
years_df = merged_df[['Year']].drop_duplicates().reset_index(drop = True).sort_values('Year')
years_df.insert(0, 'year_id', range(1, len(years_df) + 1))

In [54]:
# Datos comunes y preparación del values_df
foreign_keys = ['country_id', 'year_id']

In [55]:
# Crear la tabla de consumo conservando las llaves foráneas
consumption_merged_df = consumption_df.merge(countries_df, on=['Entity', 'Code'], how='left')
consumption_merged_df = consumption_merged_df.merge(years_df, on='Year', how='left')

consumption_merged_df['Year'] = consumption_merged_df['Year'].astype(int) #Convierte columna Year a int  
consumption_merged_df['country_id'] = consumption_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

consumption_merged_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']] = consumption_merged_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']].fillna(0)

consumption_merged_df = consumption_merged_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] + foreign_keys]

consumption_merged_df.index += 1

In [56]:
# Crear la tabla de producción conservando las llaves foráneas
production_merged_df = production_df.merge(countries_df, on=['Entity', 'Code'], how='left')
production_merged_df = production_merged_df.merge(years_df, on='Year', how='left')

production_merged_df['Year'] = production_merged_df['Year'].astype(int) #Convierte columna Year a int  
production_merged_df['country_id'] = production_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

production_merged_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']] = production_merged_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']].fillna(0)

production_merged_df = production_merged_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)'] + foreign_keys]

production_merged_df.index += 1

In [57]:
# Crear la tabla de porcentaje de energía renovable respecto al total conservando las llaves foráneas
renewable_percent_merged_df = share_energy_df.merge(countries_df, on=['Entity', 'Code'], how='left')
renewable_percent_merged_df = renewable_percent_merged_df.merge(years_df, on='Year', how='left')

renewable_percent_merged_df['Year'] = renewable_percent_merged_df['Year'].astype(int) #Convierte columna Year a int  
renewable_percent_merged_df['country_id'] = renewable_percent_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

renewable_percent_merged_df[['Renewables (% equivalent primary energy)']] = renewable_percent_merged_df[['Renewables (% equivalent primary energy)']].fillna(0)

renewable_percent_merged_df = renewable_percent_merged_df[['Renewables (% equivalent primary energy)'] + foreign_keys]
renewable_percent_merged_df.index += 1

In [58]:
# Crear la tabla de capacidad de energía solar conservando las llaves foráneas
solar_capacity_merged_df = solar_cap_df.merge(countries_df, on=['Entity', 'Code'], how='left')
solar_capacity_merged_df = solar_capacity_merged_df.merge(years_df, on='Year', how='left')

solar_capacity_merged_df['Year'] = solar_capacity_merged_df['Year'].astype(int) #Convierte columna Year a int  
solar_capacity_merged_df['country_id'] = solar_capacity_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

solar_capacity_merged_df[['Solar Capacity']] = solar_capacity_merged_df[['Solar Capacity']].fillna(0)

solar_capacity_merged_df = solar_capacity_merged_df[['Solar Capacity'] + foreign_keys]
solar_capacity_merged_df.index += 1

In [59]:
# Exportar los CSVs normalizados
countries_df.to_csv('locations.csv', index=False)
years_df.to_csv('years.csv', index=False)
continentes_df.to_csv('regions.csv',index=False)
consumption_merged_df.to_csv('consumption.csv')
production_merged_df.to_csv('production.csv')
renewable_percent_merged_df.to_csv('renewable_percent.csv')
solar_capacity_merged_df.to_csv('solar_capacity.csv')

In [60]:
"""
-- 1. Tabla para region
CREATE TABLE region (
    region_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255) NOT NULL
);

-- 2. Tabla para location
CREATE TABLE location (
    location_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255) NOT NULL,
    code VARCHAR(10),
    region_id INT NULL,
    FOREIGN KEY (region_id) REFERENCES region(region_id) ON DELETE SET NULL ON UPDATE CASCADE
);

-- 3. Tabla para year
CREATE TABLE year (
    year_id INT PRIMARY KEY AUTO_INCREMENT,
    year INT NOT NULL UNIQUE
);

-- 4. Tabla para renewable_energy_consumption
CREATE TABLE renewable_energy_consumption (
    consumption_id INT PRIMARY KEY AUTO_INCREMENT,
    bioenergy_TWh DECIMAL(10,4) NOT NULL,
    solar_TWh DECIMAL(10,4) NOT NULL,
    wind_TWh DECIMAL(10,4) NOT NULL,
    hydro_TWh DECIMAL(10,4) NOT NULL,
    location_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (location_id) REFERENCES location(location_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES year(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 5. Tabla para renewable_energy_production
CREATE TABLE renewable_energy_production (
    production_id INT PRIMARY KEY AUTO_INCREMENT,
    wind_TWh DECIMAL(10,4) NOT NULL,
    hydro_TWh DECIMAL(10,4) NOT NULL,
    solar_TWh DECIMAL(10,4) NOT NULL,
    bioenergy_TWh DECIMAL(10,4) NOT NULL,
    location_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (location_id) REFERENCES location(location_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES year(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 6. Tabla para renewable_percent
CREATE TABLE renewable_percent (
    renewable_percent_id INT PRIMARY KEY AUTO_INCREMENT,
    percent DECIMAL(5,2) NOT NULL,
    location_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (location_id) REFERENCES location(location_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES year(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 7. Tabla para solar_capacity
CREATE TABLE solar_capacity (
    solar_capacity_id INT PRIMARY KEY AUTO_INCREMENT,
    percent DECIMAL(5,2) NOT NULL,
    location_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (location_id) REFERENCES location(location_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES year(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- Ejemplos Select's
SELECT * FROM locations;
SELECT * FROM years;
SELECT * FROM regions;
SELECT * FROM consumption;
SELECT * FROM production;
SELECT * FROM renewable_percent;
SELECT * FROM solar_capacity;
"""

"\n-- 1. Tabla para region\nCREATE TABLE region (\n    region_id INT PRIMARY KEY AUTO_INCREMENT,\n    name VARCHAR(255) NOT NULL\n);\n\n-- 2. Tabla para location\nCREATE TABLE location (\n    location_id INT PRIMARY KEY AUTO_INCREMENT,\n    name VARCHAR(255) NOT NULL,\n    code VARCHAR(10),\n    region_id INT NULL,\n    FOREIGN KEY (region_id) REFERENCES region(region_id) ON DELETE SET NULL ON UPDATE CASCADE\n);\n\n-- 3. Tabla para year\nCREATE TABLE year (\n    year_id INT PRIMARY KEY AUTO_INCREMENT,\n    year INT NOT NULL UNIQUE\n);\n\n-- 4. Tabla para renewable_energy_consumption\nCREATE TABLE renewable_energy_consumption (\n    consumption_id INT PRIMARY KEY AUTO_INCREMENT,\n    bioenergy_TWh DECIMAL(10,4) NOT NULL,\n    solar_TWh DECIMAL(10,4) NOT NULL,\n    wind_TWh DECIMAL(10,4) NOT NULL,\n    hydro_TWh DECIMAL(10,4) NOT NULL,\n    location_id INT NOT NULL,\n    year_id INT NOT NULL,\n    FOREIGN KEY (location_id) REFERENCES location(location_id) ON DELETE CASCADE ON UPDATE CASC